**Zillow**

For the following, iterate through the steps you would take to create functions: Write the code to do the following in a jupyter notebook, test it, convert to functions, then create the file to house those functions.

You will have a `zillow.ipynb` file and a helper file for each section in the pipeline.

**acquire & summarize**

- Acquire data from mySQL using the python module to connect and query. 
- You will want to end with **a single dataframe**. Make sure to include: the logerror, all fields related to the properties that are available. You will end up **using all the tables in the database**.
- Be sure to do **the correct join (inner, outer, etc.)**. We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
- Only include properties with a **transaction in 2017**, and include **only the last transaction for each properity** (so no duplicate property ID's), along with zestimate error and date of transaction.
- Only include properties that include a latitude and longitude value.

**Summarize Zillow Database**

- airconditioningtype: 13 unique values
    - primary key: airconditioningtypeid


- architecturalstyletype: 27 unique values
    - primary key: architecturalstyletypeid
    
    
- buildingclasstype: 5 unique values
    - primary key: buildingclasstypeid
    
    
- heatingorsystemtype: 25 unique values
    - primary key: heatingorsystemtypeid
    
    
- predictions_2016: all the transactions in 2016 
    - No need to be joined
    
    
- predictions_2017: 77614 records in total
    - primary key: parcelid
    - 77613 records in 2017
    - 1 record in 2018
    - unique id: 77614
    - unique parcelid: 77414
    
    
- properties_2016: No need to be joined


- properties_2017: main table
    - primary key: parcelid
    
    
- propertylandusetype
    - primary key: propertylandusetypeid
    
    
- storytype: 35 unique values
    - primary key: storytypeid
    

- typeconstructiontype: 18 unqiue values
    - primary key: typeconstructiontypeid
    
    
- unique_properties: 2,985,217 rows
    - primary key: parcelid

In [1]:
import warnings
warnings.filterwarnings("ignore")
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import env, acquire

In [ ]:
df.info()